In [2]:
import os
import stardog
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode


### Specify Stardog connection details

In [3]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

Preparando o texto em turtle

In [4]:
# Prefixos
prefixos = """ @prefix ns: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix prism: <http://prismstandard.org/namespaces/1.2/basic/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schemas.talis.com/2005/address/schema#> .
@prefix status: <http://purl.org/ontology/bibo/status/> .
@prefix degrees: <http://purl.org/ontology/bibo/degrees/> .
@prefix stardog: <tag:stardog:api:> .
@base <http://www.w3.org/2002/07/owl#> .

"""

In [5]:
# Funcão que recebe os prefixos e triplas e as carrega à base de dados
def add_triplas_to_stardog(prefixos, triplas):

    # Incluindo prefixos às triplas
    triplas = prefixos + " " + triplas

    ### Connect to the Stardog database
    database_name = 'IndigenousSlavery'
    conn = stardog.Connection(database_name, **connection_details)

    conn.begin()
    conn.add(stardog.content.Raw(triplas.encode('utf-8'), 'text/turtle'))
    conn.commit() # commit the transaction

In [6]:
#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

In [7]:
# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))

### Para cada Tese, precisamos identificar a página que contem o documento inteiro, baixar o arquivo PDF para então extrair os textos das páginas.

Extraindo o texto de uma tese e carregando no grafo

In [16]:
def extratc_text_NER(tese_uri, tese_path):

    triplas = """ """

    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)

    # Iterando por cada página
    for page_number in range(number_of_pages):
        

        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  

        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
        except:
            lang = None

        # Extraindo as entidades
        if lang == 'pt':
            doc = nlp_pt(page_text)
        if lang == 'en':
            doc = nlp_en(page_text)

        persons = []
        gpes = []
        #orgs = []
        for ent in doc.ents:
            if ent.label_ == "PER": #"PERSON":
                persons.append(process_uri(ent.text))
            if ent.label_ == "LOC": #"GPE":
                gpes.append(process_uri(ent.text))
            #if ent.label_ == "ORG":
            #    orgs.append(process_uri(ent.text))


            
        tripla = """
        stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ rdf:type bibo:Excerpt.
        stardog:""" + tese_uri +  """ <http://purl.obolibrary.org/obo/BFO_0000051> stardog:""" + tese_uri + "__page_" + str(page_number + 1 ) + """.
        stardog:""" + tese_uri + "__page_" + str(page_number + 1 ) + """ bibo:pages '""" + str(page_number + 1) +  """'. 
        """
        if lang == 'pt':
            tripla = tripla + "stardog:" + tese_uri + "__page_" + str(page_number + 1 ) + " <http://purl.org/dc/terms/description> '" + page_text.replace("'","").replace(u'\\', u' ') +  "'@pt."
        if lang == 'en':
            tripla = tripla + "stardog:" + tese_uri + "__page_" + str(page_number + 1 ) + " <http://purl.org/dc/terms/description> '" + page_text.replace("'","").replace(u'\\', u' ') +  "'@en."
        
        #print(set(persons), set(gpes))
        for per in set(persons):  
            tripla = tripla + """
            stardog:""" + per + """ rdf:type foaf:Person.
            stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ <https://schema.org/mentions> stardog:""" + per + """. 
            """

        for gpe in set(gpes): 
            tripla = tripla + """
            stardog:""" + gpe + """ rdf:type <https://schema.org/Place>.
            stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ <https://schema.org/mentions> stardog:""" + gpe + """. 
            """

        triplas = triplas + " " + tripla

    add_triplas_to_stardog(prefixos, triplas)
    return


In [17]:
teses_tuplas = [("dramaturgias_negras_do_pampa__uma_analise_decolonial", 
                 "data/BDTD/Teses/abdias_nascimento/001177775.pdf"),
                 ("a_rememoracao_nas_imagens_dos_quilombos", 
                 "data/BDTD/Teses/abdias_nascimento/A rememoração nas imagens dos quilombos Dissertação Alessandra Brito.pdf"),
                 ("aquilombamento_virtual_midiatico_uma_proposta_teoricometodologica_para_o_estudo_das_midias_negras", 
                 "data/BDTD/Teses/abdias_nascimento/Aquilombamentovirtualmidiatico_Andrade_2023.pdf"),
                 ("corpokalunga_fluxos_memorialisticos_e_identitarios_em_um_defeito_de_cor_de_ana_maria_goncalves", 
                 "data/BDTD/Teses/abdias_nascimento/CorpoKalungafluxos_Silveira_2022.pdf"),
                 ("a_trajetoria_e_as_colunas_cinematograficas_de_ironides_rodrigues_para_a_marcha_19541962", 
                 "data/BDTD/Teses/abdias_nascimento/FINAL_Dissertação_Gabriel_10-04-2019_.pdf"),
                 ("escrita_pela_vivencia_memoria_em_letra_reexistindo_as_identidades", 
                 "data/BDTD/Teses/abdias_nascimento/MEMORIAL 9 DE NOVEMBRO COM FICHA CATOLOGRÁFICA (1).pdf"),
                 ("faz_parte_da_familia_contradicoes_do_trabalho_domestico_remunerado_uma_narrativa_a_partir_da_memoria_de_quartinhos_de_empregada", 
                 "data/BDTD/Teses/abdias_nascimento/tatianedemenezesbezerra.pdf"),
                 ("acoes_afirmativas_com_recorte_racial_no_ensino_superior_e_disputas_de_identidade_nacional_no_brasil", 
                 "data/BDTD/Teses/abdias_nascimento/TESE Aristeu Portela Júnior.pdf"),
                 ("a_dramaturgia_do_teatro_experimental_do_negro_ten_e_do_teatro_profissional_do_negro_tepron_corpo_e_identidades", 
                 "data/BDTD/Teses/abdias_nascimento/tese_girlene_verly_ferreira_de_carvalho_rezende.pdf")
                ]

In [18]:
for tese_uri, tese_path in teses_tuplas:
    extratc_text_NER(tese_uri, tese_path)
    print(tese_uri)

dramaturgias_negras_do_pampa__uma_analise_decolonial
a_rememoracao_nas_imagens_dos_quilombos
aquilombamento_virtual_midiatico_uma_proposta_teoricometodologica_para_o_estudo_das_midias_negras
corpokalunga_fluxos_memorialisticos_e_identitarios_em_um_defeito_de_cor_de_ana_maria_goncalves
a_trajetoria_e_as_colunas_cinematograficas_de_ironides_rodrigues_para_a_marcha_19541962
escrita_pela_vivencia_memoria_em_letra_reexistindo_as_identidades
faz_parte_da_familia_contradicoes_do_trabalho_domestico_remunerado_uma_narrativa_a_partir_da_memoria_de_quartinhos_de_empregada
acoes_afirmativas_com_recorte_racial_no_ensino_superior_e_disputas_de_identidade_nacional_no_brasil
a_dramaturgia_do_teatro_experimental_do_negro_ten_e_do_teatro_profissional_do_negro_tepron_corpo_e_identidades


In [32]:
number_of_pages

284

In [21]:
number_of_pages = len(reader.pages)
page_n = 15
page = reader.pages[page_n]
page_ant = reader.pages[page_n-1]
page_pos = reader.pages[page_n+1]
text = page.extract_text()
text_ant = page_ant.extract_text()
text_pos = page_pos.extract_text()

In [22]:
number_of_pages

446

In [23]:
print(text_ant[-400:])
print('--------')
print(text)
print('--------')
print(text_pos[:400])

 do teatro brasileiro, como é o 
caso de  Panorama do Teatro Brasileiro , de Sábato Magaldi (2004), que cita o TEN 
brevemente, e O Teatro Moderno Brasileir o, de Décio de Almeida Prado (2009), que não cita 
nenhuma delas.  
Magaldi tem seu olhar inicialmente voltado para o século XVII e diz que, desde essa 
época, alguns negros já faziam teatro, mas de forma muito marginal ou discreta (MA GALDI, 
--------
 
 
 
16 
 
 
 
   2004, p. 30) . Ness a obra do autor , a única referência acerca de Abdias Nascimento é uma 
breve passagem em que, citado ao lado de diversos outros que se aventuraram rapidamente no 
teatro, diz: “ele não alcançou forma válida para a volta às raízes raciais, na luta de protesto s 
contra injustiças do mundo branco ” (MAGALDI, 2004, p. 278) . Ainda que a iniciativa de 
Nascimento não estivesse de acordo com os princípios valorizados pelo teatro de Magaldi,  e 
de outros,  diante dos comentários de críticos e artistas da época, material h oje acessível a 
todos, o t

In [ ]:
triplas = """ """

for m in range(len(thesis_uri)):
    tripla = """
    stardog:""" + thesis_uri[m] + """ rdf:type bibo:Thesis.
    stardog:""" + thesis_uri[m] + """ terms:title '""" + titulo[m].replace("'","").replace(u'\\', u' ') +  """'.
    stardog:""" + thesis_uri[m] + """ terms:created '""" + str(ano[m]) +  """'^^xsd:gYear.
    """
    if link[m].startswith("http"):
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:repository '""" + link[m].replace("'","").replace(u'\\', u' ') + """'. """

    if referencia[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:identifier '""" + referencia[m].replace("'","").replace(u'\\', u' ') + """'. """

    if resumo[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo[m]).replace("'","").replace(u'\\', u' ') + """'@""" + resumo_lang[m] + """. """
    
    if resumo_pt[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo_pt[m]).replace("'","").replace(u'\\', u' ') + """'@pt. """

    if resumo_en[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo_en[m]).replace("'","").replace(u'\\', u' ') + """'@en. """

    triplas = triplas + " " + tripla

# stardog:""" + thesis_uri[m] + """ rdf:type owl:NamedIndividual.